# Getting Courses
---

Set username and password if you want to login automatically. Not required.

In [208]:
USERNAME = ""
PASSWORD = ""

Log into temple and wait for the request session to be created.

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
import requests
import json

# Urls that get information about courses and terms
URL_TERMS = "https://prd-xereg.temple.edu/StudentRegistrationSsb/ssb/plan/getTerms"
URL_COURSE_INFO = "https://prd-xereg.temple.edu/StudentRegistrationSsb/ssb/searchResults/searchResults"

# Post urls that are required to getting new information
URL_PLAN_MODE = "https://prd-xereg.temple.edu/StudentRegistrationSsb/ssb/term/search?mode=plan"
URL_RESET_DATAFORM = "https://prd-xereg.temple.edu/StudentRegistrationSsb/ssb/courseSearch/resetDataForm"

# Pages that you are required to visit (otherwise you cannot search for courses)
URL_PAGE_LOGIN = "https://tuportal.temple.edu"
URL_PAGE_HOME = "https://tuportal6.temple.edu/group/home"
URL_PAGE_REGISTRATION = "https://prd-xereg.temple.edu/StudentRegistrationSsb/ssb/registration"


try:
    # Initialize browser and redirect to login page
    print("Initializing browser and redirecting to temple login page.")
    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.get(URL_PAGE_LOGIN)
    driver_wait = WebDriverWait(driver, 10)

    # Fill out username and password if specified
    try:
        if len(USERNAME) > 0 and len(PASSWORD) > 0:
            print("Attempting to login automatically.")
            username_field = driver_wait.until(ec.presence_of_element_located((By.ID, "username")))
            password_field = driver_wait.until(ec.presence_of_element_located((By.ID, "password")))
            username_field.send_keys(USERNAME)
            password_field.send_keys(PASSWORD)
            login_button = driver.find_element(By.CLASS_NAME, "btn-login")
            login_button.click()
    except NameError:
        print("Username and password not defined, waiting for manual login.")
        pass

    # Detect if user has finished logging in (auto accepts trust browser)
    def finish_login(_driver):
        if button := _driver.find_elements(By.ID, "trust-browser-button"):
            button[0].click()
        return _driver.current_url == URL_PAGE_HOME

    # Redirect to self service banner page once logged in (5 minutes max to login)
    WebDriverWait(driver, 5 * 60).until(finish_login)
    print("Login complete, redirecting to registration page.")
    driver.get(URL_PAGE_REGISTRATION)

    # Click planning link once it is available
    print("Redirecting to planning page.")
    link = driver_wait.until(ec.element_to_be_clickable((By.ID, "planningLink")))
    link.click()

    # Grab required cookies from current session (required for future requests)
    print("Grabbing required session information.")
    session = requests.Session()
    session.cookies.update(
        {
            "JSESSIONID": driver.get_cookie("JSESSIONID")["value"],
            "BIGipServerprd_xereg_8180_pool": driver.get_cookie("BIGipServerprd_xereg_8180_pool")["value"],
        }
    )
    # print(json.dumps({cookie["name"]: cookie["value"] for cookie in driver.get_cookies()}, indent=4))

    # The browser is no longer required after the session is created
    print("Session created, exiting browser.")
    driver.quit()
except Exception as error:
    print("Failed to create session:", error)


def fetch(_url: str, _query_params):
    try:
        response = session.get(_url, params=_query_params)

        if response.status_code != 200:
            return False, f"Failed to retrieve the page. Status code: {response.status_code}"
        return True, response.json()
    except ValueError:
        return False, "You are not logged in."
    except Exception as error:
        return False, error.with_traceback()


def fetch_recursive(_url: str, _query_params, *, filter=None, output=False):
    _query_params["pageOffset"] = 0
    _query_params["pageMaxSize"] = 2000
    data = []

    while True:
        success, result = fetch(_url, _query_params)
        if not success:
            return success, result
        if not result["success"] or not result["data"]:
            return False, "Fetch failed. Data not found."

        if filter:
            # only append values filtered values
            data.extend(item for item in result["data"] if filter(item))
        else:
            # appened all results
            data.extend(result["data"])

        # update page offset by length of the data
        _query_params["pageOffset"] += len(result["data"])

        if output:
            print(f"Fetched data ({_query_params["pageOffset"]} out of {result["totalCount"]}).")

        if _query_params["pageOffset"] >= result["totalCount"]:
            break
    return True, data

Once the session has been created, execute the cell below to get the code for the term you are planning course for.

In [ ]:
success, result = fetch(URL_TERMS, {"offset": 0, "max": 3})

if not success:
    print("Error getting term codes:", result)
else:
    for term in result:
        print(f"CODE: {term["code"]}, TITLE: {term["description"]}")

Once you have retrieved the term code, update the `TERM_CODE` variable with the code and specify which courses you are planning to take.

In [211]:
TERM_CODE = 202503

In [213]:
SELECTED_COURSES = ["MATH2043", "IH0851", "MATH2101", "CIS1068"] # "CIS2033"
# SELECTED_COURSES = ["CIS1057", "CIS1166", "ENG0802", "CIS1001", "SCTC1001"]

# Plotting Courses
---

In [214]:
from datetime import datetime
from dateutil import parser
from itertools import product
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import numpy as np
import math
import random


class Class:
    """Class representing a class in the schedule."""

    def __init__(self, _section: dict[str, any], _day: str, _time_beg: datetime, _time_end: datetime):
        days = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]
        self.info: dict[str, any] = _section
        self.subject_course: str = _section["subjectCourse"]
        self.name: str = _section["courseTitle"]
        self.section: str = _section["sequenceNumber"]
        self.day: float = days.index(_day.lower())
        self.begin: datetime = _time_beg
        self.end: datetime = _time_end

    def __eq__(self, _other: object) -> bool:
        return (
            self.subject_course == _other.subject_course
            and self.day == _other.day
            and self.begin == _other.begin
            and self.end == _other.end
        )

    def __hash__(self) -> int:
        return hash((self.subject_course, self.day, self.begin, self.end))


class Schedule:
    """Class representing a schedule containing multiple events."""

    def __init__(self, _schedule: list[dict[str, any]]):
        self.classes: list[Class] = []
        self.add_schedule(_schedule)

    def __eq__(self, _other: object) -> bool:
        return self.classes == _other.classes

    def __hash__(self) -> int:
        return hash(tuple(self.classes))

    def add_schedule(self, _schedule: list[dict[str, any]]):
        """Adds events to the schedule from a list of courses."""
        days = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]

        for section in _schedule:
            for meeting in section["meetingsFaculty"]:
                for day in days:
                    if meeting["meetingTime"][day]:
                        time_beg = meeting["meetingTime"]["beginTime"]
                        time_beg = time_beg[:2] + ":" + time_beg[2:]
                        time_beg = parser.parse(time_beg)

                        time_end = meeting["meetingTime"]["endTime"]
                        time_end = time_end[:2] + ":" + time_end[2:]
                        time_end = parser.parse(time_end)

                        self.classes.append(Class(section, day, time_beg, time_end))

    def has_overlap(self) -> bool:
        """Checks if any events overlap within the schedule."""

        # Group events by day to check overlaps within each day
        events_by_day = {}
        for event in self.classes:
            events_by_day.setdefault(event.day, []).append(event)

        # Check for overlaps within each day
        for events in events_by_day.values():
            events.sort(key=lambda e: e.begin)  # Sort by start time
            for i in range(1, len(events)):
                if events[i - 1].end > events[i].begin:  # Overlap detected
                    return True
        return False

    def get_formatted_time(self, _hour: int) -> str:
        """Returns a formatted time string in AM/PM format."""
        label = "PM" if 12 <= _hour < 24 else "AM"
        _hour = _hour % 12 or 12  # Convert 0 to 12 for midnight
        return f"{_hour}:00 {label}"

    def get_dark_mode_colors(self) -> list:
        """Generates a list of suitable colors for dark mode from XKCD colors."""
        dark_mode_colors = []
        for color_name in mcolors.XKCD_COLORS:
            r, g, b = mcolors.to_rgb(mcolors.XKCD_COLORS[color_name])
            luminance = (r * 0.299 + g * 0.587 + b * 0.114) * 255
            if 50 <= luminance <= 80:
                dark_mode_colors.append(mcolors.XKCD_COLORS[color_name])

        dark_mode_colors.sort()
        random.seed(2024)
        random.shuffle(dark_mode_colors)
        return dark_mode_colors

    def get_range(self) -> tuple[float, float]:
        """Returns the minimum and maximum times of events in the schedule."""
        time_min = 24
        time_max = 0

        for event in self.classes:
            time_min = min(time_min, event.begin.hour + event.begin.minute / 60)
            time_max = max(time_max, event.end.hour + event.end.minute / 60)
        return time_min, time_max

    def get_schedule_time(self, time: datetime) -> float:
        """Converts a datetime object to a float representing hours left in the day."""
        return 24 - (time.hour + time.minute / 60)

    def get_classes(self) -> dict[str, str]:
        class_dict = {}
        for cls in self.classes:
            class_dict.setdefault(cls.subject_course, cls.section)
        return class_dict

    def show(self):
        """Displays the weekly schedule using matplotlib."""
        plt.figure(figsize=(20, 8))

        days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
        # times = ["{0}:00".format(hour) for hour in range(24, -1, -1)]
        times = [self.get_formatted_time(hour) for hour in range(24, -1, -1)]

        plt.xticks(np.arange(len(days)), days)
        plt.yticks(np.arange(len(times)), times)
        plt.ylabel("Time")
        plt.xlabel("Day")
        plt.title("Weekly Schedule")
        plt.style.use("dark_background")
        plt.tick_params(axis="both", which="major", pad=15)

        dark_mode_colors = self.get_dark_mode_colors()
        event_colors: dict[str, any] = {}

        if len(self.classes) > 0:
            range_min, range_max = self.get_range()
            range_min = 24 - math.floor(range_min) + 0.25
            range_max = 24 - math.ceil(range_max) - 0.25
            plt.ylim(range_max, range_min)
            # plt.ylim(0, 24)

            CELL_WIDTH = 0.98

            for event in self.classes:
                beg: float = self.get_schedule_time(event.begin)
                end: float = self.get_schedule_time(event.end)

                if event.name not in event_colors:
                    event_colors[event.name] = dark_mode_colors.pop(0)

                plt.bar(
                    event.day,
                    height=(end - beg),
                    bottom=beg,
                    width=CELL_WIDTH,
                    color=event_colors[event.name],
                    edgecolor="black",
                )
                plt.text(event.day, (beg + end) / 2, event.name, ha="center", va="center", fontsize=8)

                def to_hm(time: datetime):
                    label = "PM" if 12 <= time.hour < 24 else "AM"
                    return "{0}:{1:0>2} {2}".format(time.hour % 12 or 12, time.minute, label)

                plt.text(
                    event.day - CELL_WIDTH / 2 + 0.02, beg - 0.05, to_hm(event.begin), ha="left", va="top", fontsize=8
                )
                plt.text(
                    event.day - CELL_WIDTH / 2 + 0.02, end + 0.05, to_hm(event.end), ha="left", va="bottom", fontsize=8,fontfamily='monospace'
                )
                plt.text(
                    event.day + CELL_WIDTH / 2 - 0.02,
                    beg - 0.05,
                    f"Section: {event.section}",
                    ha="right",
                    va="top",
                    fontsize=8,
                )
                plt.text(
                    event.day + CELL_WIDTH / 2 - 0.02,
                    end + 0.05,
                    event.subject_course,
                    ha="right",
                    va="bottom",
                    fontsize=8,
                )
        plt.show()


class ScheduleCompare:
    """Class representing a function used to compare schedules for sorting."""

    def not_overlap(_s: Schedule):
        return not _s.has_overlap()

    def week_range(_s: Schedule):
        min, max = _s.get_range()
        return max - min

    def week_total(_s: Schedule):
        events_by_day = {}
        for event in _s.classes:
            events_by_day.setdefault(event.day, []).append(event)

        total_time = 0
        for events in events_by_day.values():
            events.sort(key=lambda e: e.begin)
            prev_time = events[0].begin.hour + events[0].begin.minute / 60
            curr_time = events[-1].end.hour + events[-1].end.minute / 60
            total_time += curr_time - prev_time
        return total_time

    def between_total(_s: Schedule):
        events_by_day = {}
        for event in _s.classes:
            events_by_day.setdefault(event.day, []).append(event)

        total_time = 0
        for events in events_by_day.values():
            events.sort(key=lambda e: e.begin)
            for i in range(1, len(events)):
                prev_time = events[i - 1].end.hour + events[i - 1].end.minute / 60
                curr_time = events[i].begin.hour + events[i].begin.minute / 60
                total_time += curr_time - prev_time
        return total_time


def get_schedule_combos(_course_list: list[str]) -> list[tuple]:
    all_sections = []
    for course_name in set(_course_list):
        # Refresh courses and sections (otherwise the server will cache the results)
        session.post(URL_PLAN_MODE, data={"term": TERM_CODE})
        session.post(URL_RESET_DATAFORM, data={"resetCourses": True, "resetSections": True})

        # Fetch all section info for selected courses
        success, result = fetch_recursive(
            URL_COURSE_INFO,
            {
                # Format expamples: CIS1057, SCTC1001, ENG0802
                "txt_subjectcoursecombo": course_name,
                "txt_term": TERM_CODE,
            },
            # Filter out classes that are not on the main campus or are not in person
            filter=lambda c: c["campusDescription"] == "Main" and c["instructionalMethod"] == "CLAS",
        )

        if not success:
            print("Failed to get course info.")
            return []
        else:
            # Append list of sections for each course so that we can get the cartesian product
            all_sections.append([section for section in result])

    # Get the cartesian product of all sections
    return list(product(*all_sections))

In [215]:
schedule_combos = get_schedule_combos(SELECTED_COURSES)

In [ ]:
schedules = set(Schedule(schedule) for schedule in schedule_combos)
schedules = sorted(filter(ScheduleCompare.not_overlap, schedules), key=ScheduleCompare.week_total)
print(len(schedules))

In [ ]:
from IPython.display import Markdown, display


def render_table(headers: list[str], rows: list[list[str]]):
    # Create the header row
    md = "| " + " | ".join(headers) + " |\n"
    md += "| " + " | ".join(["---"] * len(headers)) + " |\n"  # Separator row

    # Add each row of data
    for row in rows:
        md += "| " + " | ".join(row) + " |\n"

    display(Markdown(md))

plt.rcParams['font.family'] = 'Cambria'

for schedule in schedules:
    print(
        f"WEEK_RANGE: {ScheduleCompare.week_range(schedule):.2f} hrs, "
        f"WEEK_TOTAL: {ScheduleCompare.week_total(schedule):.2f} hrs, "
        f"BREAK_TOTAL: {ScheduleCompare.between_total(schedule):.2f} hrs"
    )
    render_table(["Class", "Section"], [*schedule.get_classes().items()])

    schedule.show()

In [ ]:
import ratemyprofessor

school = ratemyprofessor.get_schools_by_name("Temple University")

for s in school:
    print(s.id, s.name)

school = school[0]

if prof := ratemyprofessor.get_professor_by_school_and_name(school, "John Davies"):
    print(prof.id, prof.name)
# for school in ratemyprofessor.get_schools_by_name("Temple University"):
#     print(school.id, school.name)